# Linear Regression with categorical predictors and interaction effects
Original notebook from the [Washington University](https://faculty.washington.edu/otoomet/machinelearning-py/linear-regression.html#linear-regression-statsmodels-sklearn).

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns


## The dataset

The Titanic dataset contains information about passengers on the RMS Titanic, including whether they survived, their age, sex, and passenger class. It is widely used in data science for tasks like building predictive models to determine what factors influenced survival, with the goal of predicting a passenger's survival based on their characteristics. 

Key features 

- `survived`: Indicates whether a passenger survived (1) or not (0).
- Passenger Class (`Pclass`): The class of the ticket (1), (2), or (3) as a numerical.
- `sex`: The gender of the passenger.
- `age`: The age of the passenger, often with many missing values.
- `sibsp`: The number of siblings or spouses aboard the ship.
- `parch`: The number of parents or children aboard the ship.
- `fare`: The price of the ticket.
- `embarked`: The port where the passenger embarked (C = Cherbourg; Q = Queenstown; S = Southampton).
- `class`: The passenger class in words (first, second, third).
- `who`: variant of `sex`	
- `adult_male`: Binary boolean indicating if the passenger is an adult male.
- `deck`: The deck level, with missing values.	
- `embark_town`: The town where the passenger embarked (full city name).
- `alive`: 'yes' or 'no' indicating survival.
- `alone`: Binary boolean indicating if the passenger was alone.


## Categorical variables[](linear-regression.html#categorical-variables-and-interaction-effects)


Introduction — Categorical predictors
Categorical predictors (factors) are variables that take a limited set of discrete labels rather than measuring a continuous quantity. Examples: sex ("male"/"female"), match result ("Win","Draw","Lose"), passenger class (1st class, 2nd class, 3rd class), or city of residence. You can recognise them because:

- their pandas dtype is often object or category,
- they have only a few distinct values relative to the number of rows, or
- they use integers to represent groups (e.g., 1/2/3 for class) rather than measurements (e.g, 1/2/3 cups of coffee).

By default, regression works with numbers, not categories. But many research hypotheses are about categories, for example: 

> - $H_0$ _Italians born in Italy eat the same amount of pineapple on pizza than italians born abroad_
> - $H_1$ _Italians born in Italy eat less pineapple pizza than italians born abroad_

How do you include categorical predictors (Birth-country) in a regression model? 

The solution lies in converting the categories into numbers. Not just any numbers but numbers that __maintain the important relations between the categories__. The most common approach is to create _dummy variables_ (also called indicator variables). A dummy variable is a binary (0/1) variable that indicates whether a given observation belongs to a particular category.

For example, let's look at the Titanic dataset, which is a standard dataset when learning data science.

In [4]:
# df_with_categories = pd.read_csv("./data/titanic.csv")

# Display the DataFrame
# print("DataFrame with categorical variable:")
# df_with_categories.head()

In [8]:
import pandas as pd

# Correct file path
file_path = r"C:\Users\Ella\Data science\week 6\titanic-1.csv"

# Load the dataset
df_with_categories = pd.read_csv(file_path)

# Display the first few rows
print("DataFrame with categorical variable:")
df_with_categories.head()


DataFrame with categorical variable:


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Consider the variable `sex` with categories `female` and `male` or the variable `adult_male` with categories `True` and `False`. We can create a dummy variable where 0 (or False) indicates one category (e.g. female) and 1 (or True) indicates the other category (e.g. male).

As always, `Pandas` has some useful functions to help us with this task. The function `pd.get_dummies()` takes a DataFrame or a Series with categorical variables and returns a new DataFrame/Series with the dummy variables added.

In [9]:
# Convert categorical variable 'sex' into dummy/indicator variables
dummies = pd.get_dummies(df_with_categories['sex'], prefix='sex', drop_first=True)
df_with_dummies = pd.concat([df_with_categories, dummies], axis=1)
print("\nDataFrame with dummy categorical variable:")
df_with_dummies.head()


DataFrame with dummy categorical variable:


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,sex_male
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,True
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,False
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,True


`Statsmodels` also has pretty handy defaults: if the variable is categorical (e.g., a string), by design, it will be automatically converted to approriate dummies. For instance, `sex` is a categorical variable (with categories `female` and `male`) and hence it will be automatically converted into dummies. For instance, if we want to estimate a model 

$survived_i = \beta_0+ male_i \beta_1 + \epsilon_i$, 

we can just insert string variable `sex` into the model:


In [84]:
titanic = df_with_categories.copy()
m = smf.ols("survived ~ sex", data=titanic).fit()
print(m.summary())

                            OLS Regression Results                            
Dep. Variable:               survived   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     372.4
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           1.41e-69
Time:                        09:00:39   Log-Likelihood:                -466.09
No. Observations:                 891   AIC:                             936.2
Df Residuals:                     889   BIC:                             945.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.7420      0.023     32.171      

The line `sex[T.male]` shows the effect of `male` dummy with `female` being the reference group. This is because the categories will be ordered alphabetically, and the first (`female`) taken as the reference category.

> 
> This means that the number of coefficients estimated for a categorical variable is always one less than the number of categories. This is because one category is used as the reference category against which the other categories are compared.
> 

But sometimes the categories are not strings but have numeric labels. For instance, _male_ may be labeled as 1 and `female` as 2. Or, as is the case of _titanic_, we have numeric labels for passenger classes, another categorical variable. 

Let's analyse the survival by passenger class using a model like 

$survived_i = \beta_0 + \beta_1 * class2_i + \beta_2 * class3_i + \epsilon_i$

Class1 is missing from the formula above because it is our reference category (like _female_ above). 

In [85]:
# 1) Boolean variable used directly in formula
# True/False will be interpreted as 1/0 by the regression
m_bool = smf.ols("survived ~ adult_male", data=titanic).fit()
print("Model using boolean 'male' directly:")
print(m_bool.summary(), "\n")

Model using boolean 'male' directly:
                            OLS Regression Results                            
Dep. Variable:               survived   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     400.0
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           9.00e-74
Time:                        09:00:40   Log-Likelihood:                -456.43
No. Observations:                 891   AIC:                             916.9
Df Residuals:                     889   BIC:                             926.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------

In [86]:
# 2) pclass used as numeric (this treats pclass as a continuous numeric predictor)
m_pclass_num = smf.ols("survived ~ pclass", data=titanic).fit()
print("Model treating 'pclass' as numeric:")
print(m_pclass_num.summary(), "\n")

Model treating 'pclass' as numeric:
                            OLS Regression Results                            
Dep. Variable:               survived   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     115.0
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           2.54e-25
Time:                        09:00:41   Log-Likelihood:                -567.75
No. Observations:                 891   AIC:                             1140.
Df Residuals:                     889   BIC:                             1149.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0

`pclass` will be treated as a numeric variable, and the estimate should be interpreted as the effect per “one unit larger `pclass`”. This clearly does not make any sense. Instead, we have to tell `statsmodels` that `pclass` is a categorical variable by wrapping `pclass` into `C` function:

In [87]:
# Forcing categorical treatment with C()
# This tells the formula API to create dummy/indicator variables for each class level.
m_pclass_cat = smf.ols("survived ~ C(pclass)", data=titanic).fit()
print("Model forcing categorical treatment with C(pclass):")
print(m_pclass_cat.summary(), "\n")

Model forcing categorical treatment with C(pclass):
                            OLS Regression Results                            
Dep. Variable:               survived   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     57.96
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           2.18e-24
Time:                        09:00:42   Log-Likelihood:                -567.30
No. Observations:                 891   AIC:                             1141.
Df Residuals:                     888   BIC:                             1155.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

Now `pclass` will automatically be converted into dummies with `1` being the reference catogory. The lines `C(pclass)[T.2]` and `C(pclass)[T.3]` describe the effect of 2nd and 3rd class travel respectively (and reveal a sad story of lower survival chance for lower-class passengers).



### ✏️ Your turn
> - Try to replace pclass with adult_male in the formula above and see what happens. Compare the result with the result of the `m_bool` model above.
> - Inspect the design matrix `X` to see how variables are encoded
    > ``` 
    > y, X_bool = dmatrices("survived ~ adult_male", data=df, return_type="dataframe")
    > print("Design matrix for 'survived ~ adult_male':")
    > print(X_bool, "\n")
    > ```
> - Extract the design matrix for the model `survived ~ C(pclass)`

In [11]:
# Ensure titanic exists
titanic = df_with_categories.copy()

# Then rerun your dmatrices code
from patsy import dmatrices
import statsmodels.api as sm

y, X_bool = dmatrices("survived ~ adult_male", data=titanic, return_type="dataframe")

print("Design matrix for 'survived ~ adult_male':")
print(X_bool.head(), "\n")

model_bool_dmat = sm.OLS(y, X_bool).fit()
print(model_bool_dmat.summary())


Design matrix for 'survived ~ adult_male':
   Intercept  adult_male[T.True]
0        1.0                 1.0
1        1.0                 0.0
2        1.0                 0.0
3        1.0                 0.0
4        1.0                 1.0 

                            OLS Regression Results                            
Dep. Variable:               survived   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     400.0
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           9.00e-74
Time:                        10:33:01   Log-Likelihood:                -456.43
No. Observations:                 891   AIC:                             916.9
Df Residuals:                     889   BIC:                             926.5
Df Model:                           1                                         
Covariance Type:            nonrobust         

We may also want to introduce gender and class interaction effects to test whether the survival rate depends on class, and whether this dependence differs for mean and women. 

This can be achieved with **interaction** effects in the form male×classmale×class, the full model would look like 

$survived_i = \beta_0 + \beta_1  class2_i + \beta_2 class3_i + \beta_3 male_i + \beta_4 class2_i  male_i  + \beta_5 class3_i male_i + \epsilon_i $

In `statsmodels`, interaction effects can be intered with asterisk `*`. For instance, `x1 * x2` will include three variables: $X_1$, $X_2$, and $X_1X_2$. If any of the variables is categorical (either string or converted to categorical using `C`), the interaction effects will be inserted for all categories. Hence the Titanic class and sex example can be done as:


In [ ]:
m = smf.ols("survived ~ C(pclass)*sex", data=titanic).fit()
print(m.summary())

                            OLS Regression Results                            
Dep. Variable:               survived   R-squared:                       0.394
Model:                            OLS   Adj. R-squared:                  0.390
Method:                 Least Squares   F-statistic:                     114.9
Date:                Tue, 11 Nov 2025   Prob (F-statistic):           1.32e-93
Time:                        17:33:51   Log-Likelihood:                -399.13
No. Observations:                 891   AIC:                             810.3
Df Residuals:                     885   BIC:                             839.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

As visible from the table, all the variables from the model above are included, and the all the `pclass` categories are taken into account. The lines containing colon, `C(pclass)[T.2]:sex[T.male]` and `C(pclass)[T.3]:sex[T.male]`, are the interaction effects for _male_ and the corresponding passenger class.



---

### ✏️ Your turn

You will apply the categorical and boolean treatment ideas from the Titanic examples to the Donner Party dataset. The file is `./data/donner_party.csv`. 

#### Exercise 1

>   - Task: Create a boolean column `male = (Sex == 'M')` and fit an OLS model: `Survived ~ male`.
>   - Deliverable: the fitted summary and a one-line interpretation of the `male` coefficient (what does a positive/negative coefficient mean here?).
>   - Hint: `male` will be treated as 0/1 automatically. You can inspect the design matrix with `patsy.dmatrices` to confirm encoding.
>   - Quick check: `dmatrices('Survived ~ male', data=df, return_type='dataframe')[1]` should show an intercept and a `male` column of 0s and 1s.


In [12]:
# Ensure titanic exists
titanic = df_with_categories.copy()

# Then rerun your dmatrices code
from patsy import dmatrices
import statsmodels.api as sm

y, X_bool = dmatrices("survived ~ adult_male", data=titanic, return_type="dataframe")

print("Design matrix for 'survived ~ adult_male':")
print(X_bool.head(), "\n")

model_bool_dmat = sm.OLS(y, X_bool).fit()
print(model_bool_dmat.summary())


Design matrix for 'survived ~ adult_male':
   Intercept  adult_male[T.True]
0        1.0                 1.0
1        1.0                 0.0
2        1.0                 0.0
3        1.0                 0.0
4        1.0                 1.0 

                            OLS Regression Results                            
Dep. Variable:               survived   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     400.0
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           9.00e-74
Time:                        10:34:25   Log-Likelihood:                -456.43
No. Observations:                 891   AIC:                             916.9
Df Residuals:                     889   BIC:                             926.5
Df Model:                           1                                         
Covariance Type:            nonrobust         

Adult males were much less likely to survive than others. Non-adult males (women and children) had a predicted survival probability of about **72%**, while adult males had only about **16%**. The difference is statistically significant, and being an adult male explains roughly **31%** of the variation in survival. In short: **adult males had a much lower chance of surviving.**

#### Exercise 2: Forcing categorical treatment for marital status

>   - Task: Treat `Marital Status` as a categorical factor (values like `M`, `S`, `W`) and fit: `Survived ~ C(Marital Status)`.
>   - Deliverable: summary and identification of the reference category (which level is omitted), plus a short interpretation of one non-reference coefficient.
>   - You want S as the specific reference level: convert the column to `pd.Categorical` and set `categories=` with the desired order before fitting.
>   - Quick check: look for terms like `C(Marital Status)[T.W]` in the summary; the omitted level is the reference.



In [81]:
# Your code here

#### Exercise 3: Numeric vs categorical Age

>   - Task: Fit two models and compare: (a) `Survived ~ Age` (treats Age numeric) and (b) create age groups (e.g., `child` < 12, `youth` 12–17, `adult` 18–59, `elder` >=60), then fit `Survived ~ C(age_group)`.
>   - Deliverable: both summaries and a short note explaining why one representation may be preferable for these data.
>   - Hint: Use `pd.cut` to create `age_group`. If Age has missing values, handle them (drop or impute) and mention what you did.
>   - Quick check: compare `R-squared` (or coefficient signs) and inspect the `C(age_group)` coefficients relative to the reference group.



In [ ]:
# Your code here

#### Exercise 4: Interaction: does the effect of sex vary by marital status?

>   - Task: Fit an interaction model `Survived ~ C(Marital Status)*male` to test whether the male effect depends on marital status.
>   - Deliverable: summary and a concise interpretation of any interaction terms (e.g., `C(Marital Status)[T.S]:male`).
>   - Hint: `A * B` expands to `A + B + A:B`. When one term is categorical (`C(...)`), you will get separate interaction coefficients for each non-reference level.
>   - Quick check: are the interaction coefficients statistically different from zero? Do they change the main `male` effect's interpretation?


In [ ]:
# Your code here

#### Exercise 5: Manual dummies and verifying reference-level control

>   - Task: Use `pd.get_dummies` to create dummies for `Marital Status` (use `drop_first=True` to set a reference), build a manual design matrix (include `male` and the dummies), and fit the same model using `sm.OLS(y, X).fit()` (i.e., without formula API). Compare coefficients to the `Survived ~ C(Marital Status) + male` formula-based model.
>   - Deliverable: the manual design matrix head, the fitted summary, and a short note comparing coefficients to the formula-based model (they should match if reference levels align).
>   - Hint: Ensure you add a constant column before fitting with `sm.add_constant(X)` or include the intercept manually.

In [ ]:
# Your code here

---
## Did you know? 

#### The t-test as a linear model with categorical predictor
---

The t-test is a statistical test used to determine if there is a significant difference between the means of two groups. It is commonly used in hypothesis testing when comparing the means of two independent samples.

> - $H_0$: The means of the two groups are equal.
> - $H_1$: The means of the two groups are not equal.

for example:

> - $H_0$: People seeing a picture of a spider and seeing a real spider have the same level of anxiety
> - $H_1$: People seeing the real spider have higher anxiety than people seeing the picture

This is actually equivalent to a linear regression model with a categorical predictor with two levels.

Remember this formula?

$outcome_i = (model) + error_i$

$anxiety_i =(b_0 + b_1 TreatmentCondition_i) + \epsilon_i $

$anxiety_i = (b_0 + b_1 Picture_i ) + \epsilon_i$

This model estimates the mean anxiety level for two groups: those who only think about the spider (Picture = 0) and those who see a picture of the spider (Picture = 1).

$X_{picture} = b_0 +(b_1 × 0)$ 

$b_0 = X_{picture}$

In the `TreatmentCondition` = 1 group (those who see the picture of the spider), we have:

$X_{real} =b_0 +(b_1 ×1) X_{real} = X_{picture} +b_1$

$b_1 = X_{real} − X_{picture}$


**$b_1$ represents the difference between the group means.**

> Example borrowed from Andy Field's book "Discovering Statistics Using R"